#### We apply ccmap asa computations to a trajectory and compare numbers to gromacs equivalent located at `./CG/resarea_SASA.xvg`

###### MAX SASA from [wikipedia](https://en.wikipedia.org/wiki/Relative_accessible_surface_area) areas are in $A^2$
---
|Residue|Tien et al. 2013 (theor.)[1]|Tien et al. 2013 (emp.)[1]|Miller et al. 1987[2]|Rose et al. 1985[3]|
| --- | ---- | --- | --- | --- |
|Alanine|129.0|121.0|113.0|118.1|
|Arginine|274.0|265.0|241.0|256.0|
|Asparagine|195.0|187.0|158.0|165.5|
|Aspartate|193.0|187.0|151.0|158.7|
|Cysteine|167.0|148.0|140.0|146.1|
|Glutamate|223.0|214.0|183.0|186.2|
|Glutamine|225.0|214.0|189.0|193.2|
|Glycine|104.0|97.0|85.0|88.1|
|Histidine|224.0|216.0|194.0|202.5|
|Isoleucine|197.0|195.0|182.0|181.0|
|Leucine|201.0|191.0|180.0|193.1|
|Lysine|236.0|230.0|211.0|225.8|
|Methionine|224.0|203.0|204.0|203.4|
|Phenylalanine|240.0|228.0|218.0|222.8|
|Proline|159.0|154.0|143.0|146.8|
|Serine|155.0|143.0|122.0|129.8|
|Threonine|172.0|163.0|146.0|152.5|
|Tryptophan|285.0|264.0|259.0|266.3|
|Tyrosine|263.0|255.0|229.0|236.8|
|Valine|174.0|165.0|160.0|164.5|


##### A single function can process multiple PDB files or snapshots along a trajectory
The first positional parameter is the source of one of two possible types: a list of paths to PDB files **or** a MDAnalysis Universe object

Options are:
* `npos:Int`, the total number of elements to process
* `step: Int`, the increment between two consecutive processed elements
* `selector:str`, A simple segid selector: a string of the form "segid A or segid B ..."
* `probe:float`, the radius of the water probe in Ang.
* `hres:Boolean`, if set to *True* increase the Fibonacci grid resolution, trading accuracy for speed
* `vdw_map:Dict`, a map a VDW radii, not defined it will be guessed from the Universe object. For pdb inputs a default all atom map is used.
* `martini3:Boolean`, if set to *True* the default martini3 VDW radii map is used, overriding the previous option (usefull for martini3 PDB records).

Multithreading options are:      
* `chunk_sz:Int` the number of elements to process per task
* `ncpu:Int`, the number of workers

In [1]:
import MDAnalysis as md
from pcmap.sasa import compute_many

#### Load in the CG simulation given a tpr/gro/pdb and the corresponding xtc file 

In [2]:
## Relative long simulation, can take time to load in
ucg = md.Universe('../LSB_data/SPC_L11_CG_2.tpr', '../LSB_data/SPC_L11_CG_1_whole_skip100.xtc')


#### Run the SASA computations

In [3]:
sasa_res = compute_many(ucg,npos=50,\
           chunk_sz=5, selector="segid seg_0_A", probe=1.91, hres=True)

Generating VDW radii map from trajectory
HighResolution:: Computing SASA w/ a 1.91A radius probe over a total of 50 snapshots of 425 particles each 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.75it/s]


Compiling results over 50 snapshots


50it [00:00, 6943.52it/s]


The returned object is a `SASA_Results` with the following properties:
* `resname:np.array`
* `resID:np.array`
* `chainID:np.array`
* `sasa:np.2Darray`
* `nframe:Int`
* `nresidue:Int`

##### Compute the average and sigma SASA of each residue along the trajectory with  `SASA_Results.stats()`

In [4]:
sasa_res.stats()

[(('MET', '1', 'seg_0_A'), 180.49615112304687, 21.59135138757801),
 (('LEU', '2', 'seg_0_A'), 71.09777526855468, 31.050710406745228),
 (('SER', '3', 'seg_0_A'), 70.42540771484374, 8.279207237924606),
 (('LEU', '4', 'seg_0_A'), 106.15115112304687, 9.595877493939815),
 (('ASP', '5', 'seg_0_A'), 32.37488525390625, 15.482186278685225),
 (('PHE', '6', 'seg_0_A'), 93.177548828125, 17.758785784528175),
 (('LEU', '7', 'seg_0_A'), 88.77530212402344, 9.289112726127195),
 (('ASP', '8', 'seg_0_A'), 61.51810180664062, 8.341837869292814),
 (('ASP', '9', 'seg_0_A'), 2.303370361328125, 2.629894100299271),
 (('VAL', '10', 'seg_0_A'), 65.83273132324219, 8.562130853626122),
 (('ARG', '11', 'seg_0_A'), 209.9855694580078, 15.474510236610573),
 (('ARG', '12', 'seg_0_A'), 135.13980346679688, 13.046030005132524),
 (('MET', '13', 'seg_0_A'), 68.23903198242188, 10.397227178801273),
 (('ASN', '14', 'seg_0_A'), 80.06289306640625, 13.947693818589928),
 (('LYS', '15', 'seg_0_A'), 167.63469116210936, 17.359795853275

## Working with PDB files

#### A single PDB file

In [2]:
from pcmap.sasa import compute_from_pdb
pdb_path = "../Single_conf/AA.pdb"
n = 5

sasa_res = compute_from_pdb(pdb_path, selector="segid A", probe=1.4)

Default all atoms VDW radii map


Unknown atom  >OT1< within residue GLU
Unknown atom  >OT2< within residue GLU


In [4]:
sasa_res

{'freeASA': [{'resname': 'MET',
   'resID': '0 ',
   'chainID': 'A',
   'SASA': 201.7772216796875,
   'frac': 0.7976101636886597},
  {'resname': 'LEU',
   'resID': '1 ',
   'chainID': 'A',
   'SASA': 166.7333984375,
   'frac': 0.8342427611351013},
  {'resname': 'SER',
   'resID': '2 ',
   'chainID': 'A',
   'SASA': 101.72369384765625,
   'frac': 0.8553200960159302},
  {'resname': 'LEU',
   'resID': '3 ',
   'chainID': 'A',
   'SASA': 198.4708251953125,
   'frac': 0.8026911020278931},
  {'resname': 'ASP',
   'resID': '4 ',
   'chainID': 'A',
   'SASA': 119.7998046875,
   'frac': 0.8693397045135498},
  {'resname': 'PHE',
   'resID': '5 ',
   'chainID': 'A',
   'SASA': 154.4154052734375,
   'frac': 0.8857269287109375},
  {'resname': 'LEU',
   'resID': '6 ',
   'chainID': 'A',
   'SASA': 155.55865478515625,
   'frac': 0.845352053642273},
  {'resname': 'ASP',
   'resID': '7 ',
   'chainID': 'A',
   'SASA': 134.11749267578125,
   'frac': 0.8537240624427795},
  {'resname': 'ASP',
   'resID': 

In [7]:
sasa_res = compute_from_pdb(pdb_path, selector="segid B or segid A", probe=1.4)

Default all atoms VDW radii map


Unknown atom  >OT1< within residue GLU
Unknown atom  >OT2< within residue GLU
Unknown atom  >OT1< within residue PRO
Unknown atom  >OT2< within residue PRO


In [5]:
sasa_res

{'freeASA': [{'resname': 'MET',
   'resID': '0 ',
   'chainID': 'A',
   'SASA': 208.5496826171875,
   'frac': 0.790817141532898},
  {'resname': 'LEU',
   'resID': '1 ',
   'chainID': 'A',
   'SASA': 167.169677734375,
   'frac': 0.8338090181350708},
  {'resname': 'SER',
   'resID': '2 ',
   'chainID': 'A',
   'SASA': 106.22998046875,
   'frac': 0.8489108681678772},
  {'resname': 'LEU',
   'resID': '3 ',
   'chainID': 'A',
   'SASA': 190.93255615234375,
   'frac': 0.8101852536201477},
  {'resname': 'ASP',
   'resID': '4 ',
   'chainID': 'A',
   'SASA': 123.272705078125,
   'frac': 0.8655520081520081},
  {'resname': 'PHE',
   'resID': '5 ',
   'chainID': 'A',
   'SASA': 201.344482421875,
   'frac': 0.8509976863861084},
  {'resname': 'LEU',
   'resID': '6 ',
   'chainID': 'A',
   'SASA': 154.7955322265625,
   'frac': 0.8461107015609741},
  {'resname': 'ASP',
   'resID': '7 ',
   'chainID': 'A',
   'SASA': 105.70440673828125,
   'frac': 0.8847129344940186},
  {'resname': 'ASP',
   'resID': 

#### Several PDB files

In [6]:
from pcmap.sasa import compute_many
pdb_path = "../data/Single_conf/AA.pdb"
n = 100

sasa_res = compute_many([ pdb_path for _ in range(0, n) ],npos=50,\
           chunk_sz=5, selector="segid A", probe=1.4, hres=True)

Default all atoms VDW radii map
HighResolution:: Computing SASA w/ a 1.4A radius probe over a total of 50 pdb records  


  0%|                                                                                                                                                                  | 0/50 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

In [2]:
sasa_res.raw

[[{'freeASA': [{'resname': 'MET',
     'resID': '0 ',
     'chainID': 'A',
     'SASA': 167.74444580078125,
     'frac': 0.8317462801933289},
    {'resname': 'LEU',
     'resID': '1 ',
     'chainID': 'A',
     'SASA': 135.1763916015625,
     'frac': 0.8656150102615356},
    {'resname': 'SER',
     'resID': '2 ',
     'chainID': 'A',
     'SASA': 82.92303466796875,
     'frac': 0.8820599317550659},
    {'resname': 'LEU',
     'resID': '3 ',
     'chainID': 'A',
     'SASA': 127.75238037109375,
     'frac': 0.8729955554008484},
    {'resname': 'ASP',
     'resID': '4 ',
     'chainID': 'A',
     'SASA': 78.12359619140625,
     'frac': 0.9147940874099731},
    {'resname': 'PHE',
     'resID': '5 ',
     'chainID': 'A',
     'SASA': 57.055908203125,
     'frac': 0.9577765464782715},
    {'resname': 'LEU',
     'resID': '6 ',
     'chainID': 'A',
     'SASA': 123.09893798828125,
     'frac': 0.8776217699050903},
    {'resname': 'ASP',
     'resID': '7 ',
     'chainID': 'A',
     'SASA': 7